# Psevdo naključni generatorji in tokovne šifre

Cilji laboratorijske vaje so sledeči:
- spoznati in implementirati enostavne (ne-varne) psevdonaključne generatorje,
- implementirati razlikovalce in empirično izmeriti prednost,
- zgraditi tokovno šifro iz generatorja psevdonaključnih vrednosti,
- napasti tokovno šifro, ki uporablja slab generator psevdonaključnih vrednosti.

**POMEMBNO**

Implementacije v tej vaji so zgolj didaktične in niso primerne za uporabo v praksi. Generatorje in šifre, ki so za uporabo v praksi primerni in primerno implementirani, bomo spoznali kasneje.

In [9]:
import secrets
import hashlib
from typing import Callable

## Naloga 1: Generatorji psevdonaključni vrednosti

Implementirajte psevdonaključne generatorje po podanih navodilih.

### Naloga 1.1: Ponavljalec semena $G_{\text{Repeat}}$

Ustvarite generator, ki iz 16-bajtnega semena `seed` zapolni izhod z njegovim ponavljanjem, da ustvarimo toliko bajtov, kot jih podaja argument `n`.

In [10]:
def G_repeat(seed: bytes, n: int) -> bytes:
    g_key = bytearray()
    for i in range(n):
        g_key.append(seed[i % len(seed)])
    return bytes(g_key)

G_repeat(b'hello', 10)

b'hellohello'

In [11]:
assert G_repeat(b'abc', 1) == b'a'
assert G_repeat(b'abc', 10) == b'abcabcabca'
assert G_repeat(b'abc', 5) == b'abcab'

### Naloga 1.2: 32-bitni generator linearnih kongruenc $G_{\text{LCG-32}}$

Ustvarite linearni kongruenčni generator. Za parametre izberite:
- $a = 1 664 525$
- $c = 1 013 904 223$
- $m = 2^{32}$

Generator ustvarja 32-bitne tj. 4-bajte vrednosti. Predstavite jih kot cela števila po pravilu debelega konca. Vrnite toliko bajtov kot podaja argument `n`.

Denimo, če potrebujemo 6 naključih bajtov, to pomeni, da moramo izračunati dve vrednosti zaporedja, kot jih podaja LCG, vsako vrednost predstaviti s 4 bajti in vrniti prvih 6 od nastalih 8 bajtov. Seme naj ne bo del rezultata.

In [12]:
def G_lcg32(seed: bytes, n: int) -> bytes:
    a = 1664525
    c = 1013904223
    m = 2**32
    
    s = int.from_bytes(seed, 'big')
    g_key = bytearray()
    
    while len(g_key) < n:
        s = (a * s + c) % m
        g_key.extend(s.to_bytes(4, 'big'))
    
    return bytes(g_key[:n])
        

G_lcg32(b'dan!', 4).hex()

'e87bb10c'

In [13]:
assert G_lcg32(bytes([1, 2, 3]), 10).hex() == 'd5943f865a5b912d7c52'
assert G_lcg32(bytes([1, 2]), 3).hex() == '5607cc'
assert G_lcg32((2**16 - 1).to_bytes(4, 'big'), 30).hex() == 'a2628d528cc0cc892d18ec5489106ba340125ca6da7dcbcd5b0afac8b971'

### Generator z zgoščevanjem v števčnem načinu $G_{\text{SHA-256}}$

Spodaj podajamo implementacijo generatorja, ki velja za varnega in ga bomo uporabili kot referenco. Njegove gradnike (zgoščevalna funkcija SHA-256 in števčni način delovanja) bomo spoznali kasneje tekom semestra. Varnostnem zagotovilu navkljub te implementacije ne uporabljamo v praksi.

In [14]:
def G_sha256(seed: bytes, n: int) -> bytes:
    res = bytearray()
    i = 0
    while len(res) < n:
        res += hashlib.sha256(i.to_bytes(8, "big") + seed).digest()
        i += 1
    return bytes(res)[:n]

G_sha256(b'hello', 10)

b'\xdf\xf1\x00\x8c\xaf\x91\x86\x8f\x06\xd2'

## Naloga 2: Statistični testi (razlikovalci)

Spomnimo, statistični test ali razlikovalec je funkcija, ki iz podanega bitnega zaporedja skuša ugotoviti, ali je bilo ustvarjeno s pomočjo psevdonaključnega generatorja (vrednost `0`) ali gre za pravo naključno zaporedje (vrednost `1`).

$$A: \{0, 1\}^n \mapsto \{0, 1\}$$

### Ogrodje za empirično merjenje prednosti

Spodnja implementacija podaja način za empirično merjenje prednosti napadalca $A$ (statističnega test oz. razlikovalca) zoper generator $G$. Prednost merimo po sledečem izrazu.

$$ \text{Adv}_\text{PRG}[A, G] \approx | \underset{k \overset{R}{\leftarrow} \mathcal{K}}{\text{Pr}}[A(G(k))=1] - \underset{r \overset{R}{\leftarrow} \{0, 1\}^n}{\text{Pr}}[A(r)=1]|$$

Prednost bomo merili empirično z metodo Monte Carlo. Tekom merjenja vzorčimo generator in enakomerno porazdelitev ter vzorčene vrednosti testiramo s testom. Kot rezultat  vrnemo absolutno razliko izmerjenih relativnih frekvenc, ki značujejo, kdaj test vrne 1.

In [15]:
def adv_prg(A: Callable[[bytes], int],
            G: Callable[[bytes, int], bytes],
            n: int,
            trials: int = 400) -> float:
    # Merimo PRG
    cnt_G = 0
    for _ in range(trials):
        seed = secrets.token_bytes(16) # izberemo pravo naključno seme
        x = G(seed, n) # vzorčimo PRG
        cnt_G += A(x) # prištejemo 1, če test vrne 1, sicer 0
    pG = cnt_G / trials
    
    # Merimo na pravih naključnih vrednostih
    cnt_R = 0
    for _ in range(trials):
        r = secrets.token_bytes(n) # vzorčimo pravo naključno vrednost
        cnt_R += A(r) # prištejemo rezultat testa
    pR = cnt_R / trials
    return abs(pG - pR)

### Naloga 2.1: Implementirajte monobitnega razlikovalca

Implementirajte razlikovalca, ki primerja število ničel in enic v podanem bitnem nizu. Matematični izraz poiščite v prosojnicah s predavanj.

Bajte v bite lahko pretvorite s pomočjo nizov, kot podaja spodnji primer. V njem se 1 bajt predstavi z natanko 8 biti, tudi če so začetni biti nastavljeni na 0.

In [16]:
bajt = 49
print(f'{bajt:08b}')

00110001


In [17]:
import math
def A_monobit(data: bytes) -> int:
    one = sum(b.bit_count() for b in data)
    zero = len(data) * 8 - one
    return 1 if abs(one - zero) < 10* math.sqrt(len(data)) else 0

A_monobit(G_repeat(secrets.token_bytes(4), 4096))

0

In [18]:
for _ in range(100):
    assert adv_prg(A_monobit, G_repeat, 1024, 100) > 0.05

Izračunajmo prednost testa zoper naše tri psevdonaključne generatorje.

In [19]:
n = 512      # dolžina psevdonaključnega niza 
trials = 100 # število vzorčenj pri empiričnem merjenju prednosti 

print(f"""
Adv(A_monobit, G_repeat) ≈ {adv_prg(A_monobit, G_repeat, n, trials):.3f},
Adv(A_monobit, G_lcg32)  ≈ {adv_prg(A_monobit, G_lcg32, n, trials):.3f},
Adv(A_monobit, G_sha256) ≈ {adv_prg(A_monobit, G_sha256, n, trials):.3f}
""")


Adv(A_monobit, G_repeat) ≈ 0.510,
Adv(A_monobit, G_lcg32)  ≈ 0.000,
Adv(A_monobit, G_sha256) ≈ 0.000



### Naloga 2.2: Razlikovalec LCG

Implementirajte razlikovalca, ki zazna, da uporabljamo psevdonaključni generator LCG.

Pri tem bomo izkoristili pogosto kombinacijo parametrov, ki se uporabljajo za LCG. Kadar je modul oblike $m = 2^w$ in kadar sta parametra $a$ in $c$ liha, potem velja, da najmanj pomembni bit najmanj pomembnega bajta alternira tj. periodično skače med 0 in 1. Matematično se lahko o tem prepričamo tako, da izraz

$X_{i+1} = (a\cdot X_i + c) \bmod 2^w$

reduciramo po modulu 2 (tj. spremljamo samo najmanj pomemben bit) in dobimo:

$X_{i+1} \bmod 2 = (a \cdot X_i + c) \bmod 2$

Če vemo, da sta $a$ in $c$ liha, potem:

- $a \bmod 2 = 1$ in
- $c \bmod 2 = 1$

Vstavimo v izraz in dobimo:

$X_{i+1} \bmod 2 = (X_i \bmod 2) + 1 \bmod 2$

Slednje pomeni, da v primeru:

- ko je $X_i$ sodo (LSB=0), bo $X_{i+1}$ liho (LSB=1) in
- ko je $X_i$ liho (LSB=1), bo $X_{i+1}$ sodo (LSB=0).


Povedano drugače: če iz generatorja LCG vzamemo zaporedne vrednosti (cela 4-bajtna števila ali zgolj njihove najmanj pomembne bajte), se najmanj pomembni bit vsakega števila periodično izmenjuje med `0` in `1`. Ali še drugače: generirana števila alternirajo: sodo, liho, sodo, liho, ... Namesto naključnega niza, kjer bi se zaporedje enic in ničel naključno menjalo, dobimo ponavljajoč se vzorec s periodo 2.

Implementirajte razlikovalca, ki za podan niz bajtov preveri ali zgoraj opisana lastnost drži. Razlikovalec naj vrne 1, kadar lastnost ne drži (niz izgleda naključen) in 0, kadar lastnost drži in niz ne izgleda naključen.

In [20]:
def A_alternate_even_odd(data: bytes) -> int:    
    stevila = []
    for i in range(0, len(data) - 3, 4):
        stevilo = int.from_bytes(data[i:i+4], byteorder='big')
        stevila.append(stevilo)

    for i in range(len(stevila) - 1):
        n_lsb = stevila[i] % 2
        c_lsb = stevila[i + 1] % 2

        if n_lsb == c_lsb:
            return 1 
    return 0

A_alternate_even_odd(G_lcg32(secrets.token_bytes(4), 16))

0

In [21]:
assert A_alternate_even_odd(G_lcg32(secrets.token_bytes(4), 16)) == 0
assert A_alternate_even_odd(G_lcg32(secrets.token_bytes(4), 128)) == 0
assert A_alternate_even_odd(G_lcg32(secrets.token_bytes(4), 1024)) == 0
assert A_alternate_even_odd(G_sha256(secrets.token_bytes(4), 512)) == 1
assert A_alternate_even_odd(G_sha256(secrets.token_bytes(4), 1024)) == 1

Izračunajmo prednost implementiranega testa zoper naše tri psevdonaključne generatorje.

In [22]:
n = 512
trials = 100

print(f"""
Adv(A_alternate_even_odd, G_repeat) ≈ {adv_prg(A_alternate_even_odd, G_repeat, n, trials):.3f},
Adv(A_alternate_even_odd, G_lcg32)  ≈ {adv_prg(A_alternate_even_odd, G_lcg32, n, trials):.3f},
Adv(A_alternate_even_odd, G_sha256) ≈ {adv_prg(A_alternate_even_odd, G_sha256, n, trials):.3f}
""")


Adv(A_alternate_even_odd, G_repeat) ≈ 0.110,
Adv(A_alternate_even_odd, G_lcg32)  ≈ 1.000,
Adv(A_alternate_even_odd, G_sha256) ≈ 0.000



## Naloga 3: Tokovna šifra

V tem delu naloge bomo implementirali algoritma tokovne šifre. V pomoč naj bo spodnja funkcija.

In [23]:
def xor_bytes(s1: bytes, s2: bytes) -> bytes:
    """Izvede operacijo XOR med podanima seznamoma bajtov in vrne seznam bajtov"""
    return bytes(x ^ y for x, y in zip(s1, s2))

### Naloga 3.1: Implementirajte šifrirni algoritem

Algoritem na vhodu prejme 16-bajtni ključ (tj. seme za psevdonaključni generator), bajte čistopisa ter psevdonaključni generator.

Implementirajte šifrirni algoritem, kjer naprej s pomočjo psevdonaključnega generatorja ustvarite psevdonaključni niz, ki ga nato z operacijo XOR združite z bajti čistopisa. Kot rezultat vrnite nastale bajte tajnopisa.

In [24]:
def sc_enc(key: bytes, pt: bytes, prg=G_lcg32) -> bytes:
    keystream = prg(key, len(pt))
    ct = xor_bytes(pt, keystream)
    return ct
    
sc_enc(b'my_key', b'hello world!')

b'\xa0G<\xe8\x80\x05\xd5|A\xaa\xa4w'

In [25]:
assert sc_enc(b'my_key', b'hello world!', G_sha256).hex() == '986900b165276d744dcce380'
assert sc_enc(b'my_key', b'hello world!', G_repeat).hex() == '051c33070a591a162d070158'
assert sc_enc(b'my_key', b'hello world!', G_lcg32).hex() == 'a0473ce88005d57c41aaa477'

### Naloga 3.2: Implementirajte dešifrirni algoritem

Algoritem na vhodu prejme 16-bajtni ključ (tj. seme za psevdonaključni generator), bajte tajnopisa ter psevdonaključni generator.

Implementirajte dešifrirni algoritem, kjer naprej s pomočjo psevdonaključnega generatorja ustvarite psevdonaključni niz, ki ga nato z operacijo XOR združite z bajti tajnopisa. Kot rezultat vrnite nastale bajte čistopisa.

In [26]:
def sc_dec(key: bytes, ct: bytes, prg=G_lcg32) -> bytes:
    key_s = prg(key, len(ct))
    pt = xor_bytes(ct, key_s)
    return pt

sc_dec(b'my_key', bytes.fromhex('986900b165276d744dcce380'), G_sha256)

b'hello world!'

In [27]:
for prg in [G_lcg32, G_sha256, G_repeat]:
    assert sc_dec(b'my long key', sc_enc(b'my long key', b'hello stream ciphers!', prg), prg) == b'hello stream ciphers!'

### Naloga 3.3: Napad na tokovno šifro

Sedaj bomo napadli tokovno šifro, ki uporablja slab generator psevdonaključnih vrednosti. 

**Situacija.** Kot napadalec smo prestregli tajnopis, ki ga je Ana poslala na spletni strežnik. Gre torej za spletni zahtevek, ki je bil poslan po metodi `POST` na naslov `https://spletna-stran.si/login`. Uporabljen je bil protokol `HTTP/1.1`. Naš cilj je dešifrirati celoten spletni zahtevek, predvsem pa v njem navedena uporabniško ime in geslo.

**Dodatne informacije.** Ana in spletni strežnik šifrirata promet s tokovno šifro, kot psevdonaključni generator pa uporabljata LCG z 32-bitnim stanjem. Ključa (semena) ne poznamo, vemo pa nekaj o strukturi sporočila (oz. uporabljenemu protokolu) ter poznamo parametre generatorja, ki so sledeči (enaki kot zgoraj):

- $a = 1 664 525$
- $c = 1 013 904 223$
- $m = 2^{32}$

Dešifrirajte spletni zahtevek, ki je podan v datoteki `data/http-post-req.bin`.

In [28]:
with open('data/http-post-req.bin', 'rb') as h:
    ct = h.read()

In [29]:
# Nadaljujte tu
# Xn+1 = ( aXn + c ) mod m

# Preverimo ranjivosti:
print(f"Ponavljanje zaporedja sod, lih: {A_alternate_even_odd(ct)}")
print(f"Primerjanje ničel in enic: {A_monobit(ct)}")

partKey = b"POST /login HTTP/1.1\r\n"   # Tku vemo da ta oblika sporočila zgleda
seed = bytearray([ct[i] ^ partKey[i] for i in range(len(partKey))]) # Key = ct XOR pt
X0 = seed[0:4]  # Vzamemo prve 4 byte (seed) in na podlagi njih lahko izračunamo za naslednje 4, ...

pt = sc_dec(X0, ct[4:], G_lcg32) # Dešifriramo

print(pt.decode('utf-8'))





Ponavljanje zaporedja sod, lih: 1
Primerjanje ničel in enic: 1
 /login HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Content-Length: 29
Content-Type: application/x-www-form-urlencoded; charset=utf-8
Host: spletna-stran.si
User-Agent: HTTPie/3.2.4

username=ana&password=hunter2
